# Lets save our data as a huggingface dataset, so it's quick to reuse



In [36]:
# import your package
%load_ext autoreload
%autoreload 2

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="<level>{message}</level>", level="INFO")

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import numpy as np


from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import pickle
import hashlib
from pathlib import Path

import transformers
from datasets import Dataset, DatasetInfo, load_from_disk, load_dataset


from tqdm.auto import tqdm
import os, re, sys, collections, functools, itertools, json

transformers.__version__


'4.31.0'

In [38]:
from src.models.load import load_model
from src.datasets.load import ds2df
from src.datasets.load import rows_item
from src.datasets.batch import batch_hidden_states
# from src.datasets.scores import choice2ids, scores2choice_probs

# Params

In [39]:
# Params
BATCH_SIZE = 1  # None # None means auto # 6 gives 16Gb/25GB. where 10GB is the base model. so 6 is 6/15
USE_MCDROPOUT = True

from src.extraction.config import ExtractConfig

cfg = ExtractConfig(
    # model="HuggingFaceH4/starchat-beta",
    # model="TheBloke/CodeLlama-13B-Instruct-fp16", # too large!
    model="WizardLM/WizardCoder-3B-V1.0",
    # model="WizardLM/WizardCoder-1B-V1.0",
    # model="WizardLM/WizardCoder-Python-7B-V1.0", # too large!
    datasets = [
        "imdb", 
        # "amazon_polarity",
        # "truthful_qa",
                #"super_glue:boolq", "EleutherAI/truthful_qa_mc", "EleutherAI/arithmetic", "NeelNanda/counterfact-tracing"
                ],
    max_examples=(12, 12),
)
cfg

ExtractConfig(model='WizardLM/WizardCoder-3B-V1.0', datasets=['imdb'], data_dirs=(), int4=True, max_examples=(12, 12), num_shots=2, num_variants=-1, layers=(), seed=42, token_loc='last', template_path=None)

# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large coding ones might be best for lying.

In [ ]:
from src.models.load import verbose_change_param, AutoConfig, AutoTokenizer, AutoModelForCausalLM

def load_model(model_repo = "HuggingFaceH4/starchat-beta"):
    # see https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/starchat.py
    model_options = dict(
        device_map="auto",
        # load_in_8bit=True,
        # load_in_4bit=True,
        torch_dtype=torch.float16, # note because datasets pickles the model into numpy to get the unique datasets name, and because numpy doesn't support bfloat16, we need to use float16
        # use_safetensors=False,
    )

    config = AutoConfig.from_pretrained(model_repo, use_cache=False)
    verbose_change_param(config, 'use_cache', False)
    
    tokenizer = AutoTokenizer.from_pretrained(model_repo)
    verbose_change_param(tokenizer, 'pad_token_id', 0)
    verbose_change_param(tokenizer, 'padding_side', 'left')
    verbose_change_param(tokenizer, 'truncation_side', 'left')
    
    model = AutoModelForCausalLM.from_pretrained(model_repo, config=config, **model_options)

    return model, tokenizer

model, tokenizer = load_model(cfg.model)

KeyboardInterrupt: 

# Scratch

In [ ]:
token_y = tokenizer(' True').input_ids
token_n = tokenizer(' Fakse').input_ids

In [ ]:
# from src.datasets.dropout import enable_dropout

# def get_gradients(model, outputs,token_y, token_n):
#     model.zero_grad()
#     score_y = outputs["scores"][:, token_y]
#     score_n = outputs["scores"][:, token_n]
#     pred = score_y - score_n
#     loss = F.mse_loss(pred, -pred)
#     loss.backward()
#     ps = model.named_parameters()
#     grads = {n:g.grad.cpu() for n,g in ps if g.grad is not None}
#     model.zero_grad()
#     model.eval()
#     return grads

    
# input_text ="the sky is blue is a statement which is "
# truncation_length = 999
# t = tokenizer(
#     input_text,
#     return_tensors="pt",
#     add_special_tokens=True,
#     padding='max_length', max_length=truncation_length, truncation=True, return_attention_mask=True,
# )
# input_ids = t.input_ids.to(model.device)
# attention_mask = t.attention_mask.to(model.device)

# model_inputs = model.prepare_inputs_for_generation(input_ids=input_ids, attention_mask=attention_mask, use_cache=False)
# outputs = model.forward(
#     **model_inputs,
#     return_dict=True,
#     output_hidden_states=True,
# )
# outputs["scores"] = outputs.logits[:, -1, :]

# grads_all = get_gradients(model, outputs,token_y, token_n)

In [ ]:
# outputs["scores"].shape
# model

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# outputs['hidden_states'][0][0, -1].shape

In [ ]:
# import re
# p = ".+mlp.c_proj.weight" # get the last weight of each layer (ignore bias)
# grads = torch.stack([g.mean(1).float() for k,g in grads_all.items() if re.match(p, k)]).numpy()
# grads.shape

In [ ]:
# p = ".+mlp.c_proj.bias" # get the last weight of each layer (ignore bias)
# grads = torch.stack([g.float() for k,g in grads_all.items() if re.match(p, k)]).numpy()
# grads.shape

In [ ]:
# plt.hist(grads.flatten(), 55, density=True)
# plt.title('grads')

In [ ]:
# hs = outputs['hidden_states'][5][0, -1].cpu().float().detach()
# plt.hist(hs, 55, density=True)
# plt.title('hidden_states')

# Load Dataset

In [ ]:
from itertools import chain, islice
from datasets import Dataset
import functools
# from datasets.arrow_dataset import Dataset
from src.prompts.prompt_loading import load_prompts

@functools.lru_cache()
def count_tokens(s):
    return len(tokenizer(s).input_ids)

def answer_len(answer_choices: list):
    a = count_tokens(answer_choices[0])
    b = count_tokens(answer_choices[1])
    return max(a, b)


def sample_n_true_y_false_prompts(prompts, num_truth=1, num_lie=1, seed=42):
    """sample some truth and some false"""
    df = pd.DataFrame(prompts)
    
    # restrict to template where the choices are a single token
    m = df.answer_choices.map(answer_len)<=2
    df = df[m]
    df = pd.concat([
        df.query("instructed_to_lie==True").sample(num_truth, random_state=seed),
        df.query("instructed_to_lie==False").sample(num_lie, random_state=seed)])
    return df.to_dict(orient="records")


# for ds_name in ds_names:
#     for split_type in ["train", "test"]:
        
# loop through all prompts in this dataset
ds_names = cfg.datasets
split_type = "train"

ds_name = ds_names[0]
prompt_ds = load_prompts(
    ds_name,
    num_shots=cfg.num_shots,
    split_type=split_type,
    template_path=cfg.template_path,
    seed=cfg.seed,
)

# for each example, sample true and false
N = cfg.max_examples[split_type!="train"]
g = map(lambda r: sample_n_true_y_false_prompts(r[1], seed=r[0]+cfg.seed), enumerate(prompt_ds))

# and combine them into one big list
g = chain.from_iterable(g) 
prompt_ds2 = list(tqdm(islice(g, N), total=N))

# convert to hugginface dataset
dataset = Dataset.from_list(prompt_ds2)
dataset

  0%|          | 0/12 [00:00<?, ?it/s]

Extracting 13 variants of each prompt


Dataset({
    features: ['ds_string', 'example_i', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name'],
    num_rows: 12
})

In [ ]:
# b = next(iter(prompt_ds))
# b
# sample_n_true_y_false_prompts(b)

# add choice tokens to dataset

## Format prompts

The prompt is the thing we most often have to change and debug. So we do it explicitly here.

We do it as transforms on a huggingface dataset.

In this case we use multishot examples from train, and use the test set to generated the hidden states dataset. We will test generalisation on a whole new dataset.


In [ ]:
from src.datasets.scores import scores2choice_probs
from src.datasets.scores import choice2id, choice2ids

def row_choice_ids(r):
    return choice2ids([[c] for c in r['answer_choices']], tokenizer)




In [ ]:
ds = (
    dataset
    .map(
        lambda ex: tokenizer(
            ex["question"], padding="max_length", max_length=600, truncation=True, add_special_tokens=True,
            # return_tensors="pt",
            return_attention_mask=True,
        ),
        batched=True,
    )
    .map(
        lambda r: {"prompt_truncated": tokenizer.batch_decode(r["input_ids"])},
        batched=True,
    )
    .map(lambda r: {'choice_ids': row_choice_ids(r)})
)
ds

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Dataset({
    features: ['ds_string', 'example_i', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'input_ids', 'attention_mask', 'prompt_truncated', 'choice_ids'],
    num_rows: 12
})

## Save as Huggingface Dataset

In [ ]:
# get dataset filename
sanitize = lambda s:s.replace('/', '').replace('-', '_') if s is not None else s

dataset_name = f"{sanitize(cfg.model)}_{ds_name}_{split_type}_{N}"
dataset_name
f = f"../.ds/{dataset_name}"
print(f)

../.ds/WizardLMWizardCoder_3B_V1.0_imdb_train_12


In [ ]:
# ds[10]

In [ ]:
gen_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    data=ds,
    batch_size=BATCH_SIZE,
)
gen_kwargs

{'model': GPTBigCodeForCausalLM(
   (transformer): GPTBigCodeModel(
     (wte): Embedding(49153, 2816)
     (wpe): Embedding(8192, 2816)
     (drop): Dropout(p=0.1, inplace=False)
     (h): ModuleList(
       (0-35): 36 x GPTBigCodeBlock(
         (ln_1): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
         (attn): GPTBigCodeAttention(
           (c_attn): Linear(in_features=2816, out_features=3072, bias=True)
           (c_proj): Linear(in_features=2816, out_features=2816, bias=True)
           (attn_dropout): Dropout(p=0.1, inplace=False)
           (resid_dropout): Dropout(p=0.1, inplace=False)
         )
         (ln_2): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
         (mlp): GPTBigCodeMLP(
           (c_fc): Linear(in_features=2816, out_features=11264, bias=True)
           (c_proj): Linear(in_features=11264, out_features=2816, bias=True)
           (act): PytorchGELUTanh()
           (dropout): Dropout(p=0.1, inplace=False)
         )
       )
     )
   

In [ ]:
# ds['choice_ids']

In [ ]:
info_kwargs = dict(cfg=cfg, ds_name=ds_name, split_type=split_type)

model.cuda()

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49153, 2816)
    (wpe): Embedding(8192, 2816)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPTBigCodeBlock(
        (ln_1): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=2816, out_features=3072, bias=True)
          (c_proj): Linear(in_features=2816, out_features=2816, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=2816, out_features=11264, bias=True)
          (c_proj): Linear(in_features=11264, out_features=2816, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2816,), eps=

In [41]:
ds1 = Dataset.from_generator(
    generator=batch_hidden_states,
    info=DatasetInfo(
        description=f"kwargs={info_kwargs}",
        config_name=f,
    ),
    gen_kwargs=gen_kwargs,
).with_format("numpy")

ds1

## Add labels

For our probe. Given next_token scores (logits) we take only the subset the corresponds to our negative tokens (e.g. False, no, ...) and positive tokens (e.g. Yes, yes, affirmative, ...).


In [ ]:
from src.datasets.scores import choice2id, choice2ids

In [ ]:
def expand_choices(choices: List[str]) -> List[str]:
    """expand out choices by adding versions that are upper, lower, whitespace, etc"""
    new = []
    for c in choices:
        new.append(c)
        new.append(c.upper())
        new.append(c.capitalize())
        new.append(c.lower())
    return set(new)


left_choices = list(r[0] for r in ds1['answer_choices'])+['no', 'false', 'negative', 'wrong']
right_choices = list(r[1] for r in ds1['answer_choices'])+['yes', 'true', 'positive', 'right']
left_choices, right_choices = expand_choices(left_choices), expand_choices(right_choices)
expanded_choices = [left_choices, right_choices]
expanded_choice_ids = choice2ids(expanded_choices, tokenizer)
expanded_choices

In [ ]:
from src.datasets.scores import scores2choice_probs

In [ ]:
# this is just based on pairs for that answer...
add_txt_ans0 = lambda r: {'txt_ans0': tokenizer.decode(r['scores0'].argmax(-1))}
add_txt_ans1 = lambda r: {'txt_ans1': tokenizer.decode(r['scores1'].argmax(-1))}

def row_choice_ids(r, tokenizer):
    return choice2ids([[c] for c in r['answer_choices']], tokenizer)

# Either just use the template choices
add_ans = lambda r: scores2choice_probs(r, row_choice_ids(r))

# Or all expanded choices
add_ans_exp = lambda r: scores2choice_probs(r, expanded_choice_ids, prefix="expanded_")

ds3 = (
    ds1
    .map(add_ans)
    .map(add_ans_exp)
    .map(add_txt_ans0)
    .map(add_txt_ans1)
)
ds3

## Save to disk

In [ ]:
ds3.save_to_disk(f)
f

# QC

In [ ]:
ds4 = load_from_disk(f)
ds4

In [ ]:

# QC, check which answers are most common
common_answers = pd.Series(ds4['txt_ans1']).value_counts()
display('Remember it should be binary. Found common LLM answers:', common_answers)


current_choices = set(list(chain(*ds4['answer_choices'])))
unexpected_answers = set(common_answers.head(10).index)-current_choices
if len(unexpected_answers):
    logger.warning(f'found unexpected answers: {unexpected_answers}. You may want to add them to class2choices')
    
mean_prob = ds4['choice_probs1'].sum(-1).mean()
print('mean_prob', mean_prob)
assert ds4['choice_probs1'].sum(-1).mean()>0.4, f"""
Our choices should cover most common answers. But they accounted for a mean probability of {mean_prob:2.2%} (should be >40%). 

To fix this you might want to improve your prompt or add to your choices
"""

In [ ]:
df = ds2df(ds4)
df

In [ ]:
# QC check accuracy
# it should manage to lie some of the time when asked to lie. Many models wont lie unless very explicitly asked to, but we don't want to do that, we want to leave some ambiguity in the prompt

d = df.query('instructed_to_lie==True')
acc = (d.label_instructed==d.llm_ans).mean()
print(f"when the model tries to lie... we get this acc {acc:2.2f}")
assert acc>0.1, f"should be acc>0.1 but is acc={acc}"

### QC stats

In [ ]:
def stats(df):
    return dict(
        acc=(df.llm_ans == df.label_instructed).mean(),
        n=len(df),
    )
    
def col2statsdf(df, group):
    return pd.DataFrame(df.groupby(group).apply(stats).to_dict()).T
    
    
print("how well does it do the simple task of telling the truth, for each template")
col2statsdf(df.query('sys_instr_name=="truth"'), 'template_name')

In [ ]:
print("how well does it complete the task for each prompt")
# of course getting it to tell the truth is easy, but how effective are the other prompts?
col2statsdf(df, 'sys_instr_name')

### QC view row

In [ ]:
# QC by viewing a row
r = ds4[0]
print(r['prompt_truncated'])
print(r['txt_ans1'])

# QC: generation

Let's a quick generation, so we can QC the output and sanity check that the model can actually do the task

In [ ]:
# r = ds[2]
# q = r["prompt_truncated"]

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )
# sequences = pipeline(
#     q.lstrip('<|endoftext|>'),
#     max_length=100,
#     do_sample=False,
#     return_full_text=False,
#     eos_token_id=tokenizer.eos_token_id,
# )

# for seq in sequences:
#     print("-" * 80)
#     print(q)
#     print("-" * 80)
#     print(f"`{seq['generated_text']}`")
#     print("-" * 80)
#     print("label", r['label'])


# QC: linear probe

In [ ]:
df

In [ ]:
hs = ds4['hs1']-ds4['hs0']
X = hs.reshape(hs.shape[0], -1)
y = (ds4['ans1'] - ds4['ans0'])>0

In [ ]:
true_switch_sign = ds4['label_true']*2-1
# true_switch_sign = ds4['true'][:, 0]*2-1
y = ((ds4['ans1'] - ds4['ans0']) * true_switch_sign) > 0

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# # Define X and y
# X = dm.hs1-dm.hs2
# y = dm.y>0

# split
n = len(y)
max_rows = 1000
print('split size', n//2)
X_train, X_test = X[:n//2], X[n//2:]
y_train, y_test = y[:n//2], y[n//2:]
X_train = X_train[:max_rows]
y_train = y_train[:max_rows]
X_test = X_test[:max_rows]
y_test = y_test[:max_rows]

# scale
scaler = RobustScaler()
scaler.fit(X_train)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)
print('lr')

lr = LogisticRegression(class_weight="balanced", penalty="l2", max_iter=380)
lr.fit(X_train2, y_train>0)

In [ ]:
print("Logistic cls acc: {:2.2%} [TRAIN]".format(lr.score(X_train2, y_train>0)))
print("Logistic cls acc: {:2.2%} [TEST]".format(lr.score(X_test2, y_test>0)))